# Tensorflow Object Detection API and AWS Sagemaker

In [ ]:
%%capture
%pip install tensorflow_io sagemaker -U

In [ ]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework
import subprocess

In [ ]:
role = sagemaker.get_execution_role()
print(role)

In [ ]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project/logs/'

In [ ]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [ ]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

In [ ]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

In [ ]:
models_config = {
    'efficientdet': {
        'url': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz',
        'tar_name': 'efficientdet_d1_coco17_tpu-32',
        'pipeline': 'pipeline_efficientdet.config',
        'job_name': 'tf2-efficientdet',
        'tensorboard_s3': "s3://object-detection-project/logs/efficientdet/"
    },
    'mobilenet': {
        'url': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
        'tar_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8',
        'pipeline': 'pipeline_mobilenet.config',
        'job_name': 'tf2-mobilenet',
        'tensorboard_s3': "s3://object-detection-project/logs/mobilenet/"
    },
    'resnet50': {
        'url': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz',
        'tar_name': 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8',
        'pipeline': 'pipeline_resnet50.config',
        'job_name': 'tf2-resnet50',
        'tensorboard_s3': "s3://object-detection-project/logs/resnet50/"
    }
}


In [ ]:

trained_models = {}

for model_name, config in models_config.items():
    print(f"\n{'='*60}")
    print(f"start training: {model_name.upper()}")
    print(f"{'='*60}\n")

    os.makedirs("/tmp/checkpoint", exist_ok=True)
    os.makedirs("source_dir/checkpoint", exist_ok=True)

    tar_path = f"/tmp/{model_name}.tar.gz"

    tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=config['tensorboard_s3'],
    container_local_output_path='/opt/training/'
    )

    subprocess.run([
        "wget", "-O", tar_path, config["url"]
    ], check=True)

    subprocess.run([
        "tar", "-zxvf", tar_path,
        "--strip-components", "2",
        "--directory", "source_dir/checkpoint",
        f"{config['tar_name']}/checkpoint"
    ], check=True)

    estimator = CustomFramework(
        role=role,
        image_uri=container,
        entry_point='run_training.sh',
        source_dir='source_dir/',
        hyperparameters={
            "model_dir": "/opt/training",
            "pipeline_config_path": config["pipeline"],
            "num_train_steps": "2000",
            "sample_1_of_n_eval_examples": "1"
        },
        instance_count=1,
        instance_type='ml.g5.xlarge',
        tensorboard_output_config=tensorboard_output_config,
        disable_profiler=True,
        base_job_name=config["job_name"]
    )

    estimator.fit(inputs)
    print(f"Finished training {model_name}")

print("All models trained successfully!")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir_spec \
    efficientdet: s3://object-detection-project/logs/efficientdet/, \
    mobilenet: s3://object-detection-project/logs/mobilenet/, \
    resnet50: s3://object-detection-project/logs/resnet50/

In [ ]:
# your write-up goes here.